In [22]:
import pandas as pd
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from nltk.corpus import stopwords
import string  # Ajout de la bibliothèque pour la gestion de la ponctuation
#import re




In [23]:
# Téléchargez les ressources nécessaires pour NLTK
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('popular')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ziedj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ziedj\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\ziedj\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\ziedj\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\ziedj\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\ziedj\AppData\Roaming\nltk_dat

True

In [33]:
# Créez un exemple de dataset
data = {
    'text': ["Hello&&&, !this is a spam message going.", "Check out this amazing offer!", "Not a spam, just a regular message.","This is the 2nd time we have tried 2 contact u"]
    ,'label': ['spam', 'spam', 'ham','spam']
}

df = pd.DataFrame(data)
df


,text,label
0,"Hello&&&, !this is a spam message going.",spam
1,Check out this amazing offer!,spam
2,"Not a spam, just a regular message.",ham
3,This is the 2nd time we have tried 2 contact u,spam


In [34]:
# Lemmatization et nettoyage du texte avec suppression de la ponctuation et des stopwords
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    # Suppression de la ponctuation
    text = ''.join([char for char in text if char not in string.punctuation])
    #word.isalpha() pour laisser que des lettres 
    # Suppression des stopwords et lemmatization
    tokens = word_tokenize(text)
    lemmatized_words = [lemmatizer.lemmatize(word.lower()) for word in tokens if word.isalpha() and word.lower() not in stop_words]
    return ' '.join(lemmatized_words)

df['processed_text'] = df['text'].apply(preprocess_text)

In [35]:
df

,text,label,processed_text
0,"Hello&&&, !this is a spam message going.",spam,hello spam message going
1,Check out this amazing offer!,spam,check amazing offer
2,"Not a spam, just a regular message.",ham,spam regular message
3,This is the 2nd time we have tried 2 contact u,spam,time tried contact u


In [39]:
"""def preprocess_text1(text):
    # Suppression de la ponctuation et lemmatization
    tokens = [lemmatizer.lemmatize(word.lower()) for word in re.split(r'\W+', text) if word.isalpha()]
    # Suppression des stopwords
    lemmatized_words = [word for word in tokens if word not in stop_words]
    return ' '.join(lemmatized_words)

df['processed_text1'] = df['text'].apply(preprocess_text1)
df"""

"def preprocess_text1(text):\n    # Suppression de la ponctuation et lemmatization\n    tokens = [lemmatizer.lemmatize(word.lower()) for word in re.split(r'\\W+', text) if word.isalpha()]\n    # Suppression des stopwords\n    lemmatized_words = [word for word in tokens if word not in stop_words]\n    return ' '.join(lemmatized_words)\n\ndf['processed_text1'] = df['text'].apply(preprocess_text1)\ndf"

In [ ]:
""" def clean (text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = " ".join([ps.stem(word) for word in tokens if word not in stopwords])
    
    return text """

In [41]:
# Division des données en ensembles d'entraînement et de test
#X=df['processed_text']
#y= df['label']
X_train, X_test, y_train, y_test = train_test_split(df['processed_text'], df['label'], test_size=0.2, random_state=42)



In [42]:
# Vectorisation du texte avec TF-IDF
tfidf = TfidfVectorizer()
tfidf.fit(X_train,y_train)
X_train_tfidf = tfidf.fit_transform(X_train)
X_test_tfidf=tfidf.transform(X_test)


In [43]:
tfidf.get_feature_names_out()

array(['contact', 'going', 'hello', 'message', 'regular', 'spam', 'time',
       'tried'], dtype=object)

In [45]:
X_test_tfidf.toarray()

array([[0., 0., 0., 0., 0., 0., 0., 0.]])

In [46]:
X_test_tfidf.toarray().shape

(1, 8)

In [47]:
sparse_matrix = tfidf.transform(X_test)
sparse_matrix

<1x8 sparse matrix of type '<class 'numpy.float64'>'
	with 0 stored elements in Compressed Sparse Row format>

In [48]:
# Entraînement d'un modèle simple (Naive Bayes pour l'exemple)
model = MultinomialNB()
model.fit(X_train_tfidf, y_train)



MultinomialNB()

In [49]:
# Prédictions sur l'ensemble de test
y_pred = model.predict(X_test_tfidf)



In [50]:
y_pred

array(['spam'], dtype='<U4')

In [51]:
# Évaluation du modèle
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy}')



Accuracy: 1.0


In [52]:
# Rapport de classification
classification_rep = classification_report(y_test, y_pred)
print('Classification Report:\n', classification_rep)

Classification Report:
               precision    recall  f1-score   support

        spam       1.00      1.00      1.00         1

    accuracy                           1.00         1
   macro avg       1.00      1.00      1.00         1
weighted avg       1.00      1.00      1.00         1

